In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%load_ext autoreload
%autoreload 2

In [2]:
from requests_toolbelt.multipart import decoder
import requests
import io
import os
import json
from PIL import Image

In [3]:
# this example notebook will process the first 8 images with filenames ending in .jpg or .JPG in this directory
sample_input_dir = '../sample_input/test_images'

ip_address = ''
port = ''
base_url = 'http://{}:{}/v1/camera-trap/sync/'.format(ip_address, port)  # insert the IP address of the API
base_url

# override with production URL
# base_url = 'https://aiforearth.azure-api.net/camera-trap/'
# base_url

'http://boto.eastus.cloudapp.azure.com:6002/v1/camera-trap/sync/'

In [4]:
# copy API key if testing production endpoint
API_KEY = ''

## Find out the version of the camera trap model used
Good endpoint to check that it is alive

In [5]:
# re-using the headers field messes with the requests connection...
headers = {
    'Ocp-Apim-Subscription-Key': API_KEY
}

version_info = requests.get(base_url + 'detector_model_version', headers=headers)
version_info.text

'v4.1.0'

## Calling the API

In [13]:
%%time

# re-using the headers field messes with the requests connection...
headers = {
    'Ocp-Apim-Subscription-Key': API_KEY
}

num_images_to_upload = 2  # the /detect endpoint currently accepts a maximum of 8 images

detection_confidence = 0.8  # a value from 0 to 1
render_boxes = True  # True to have returned result contain annotated image files; in any case a json with result will be returned

params = {
    'confidence': detection_confidence,
    'render': render_boxes
}

files = {}

num_images = 0
open_files = []
for i, image_name in enumerate(sorted(os.listdir(sample_input_dir))):
    if not image_name.lower().endswith('.jpg'):
        continue
    
    if num_images >= num_images_to_upload:
        break
    else:
        num_images += 1
    
    img_path = os.path.join(sample_input_dir, image_name)
    fd = open(img_path, 'rb')
    open_files.append(fd)
    files[image_name] = (image_name, fd, 'image/jpeg')

print('number of images to send:', len(files))

number of images to send: 2
CPU times: user 531 µs, sys: 961 µs, total: 1.49 ms
Wall time: 1.55 ms


In [14]:
r = requests.post(base_url + 'detect', 
                  params=params,
                  files=files, headers=headers)
print('response: ', r.status_code)

if not r.ok:
    r.reason
    r.text
r.elapsed.total_seconds()

for fd in open_files:
    fd.close()

response:  200


4.219137

In [8]:
r.headers

{'Server': 'gunicorn/20.0.4', 'Date': 'Mon, 01 Jun 2020 21:20:02 GMT', 'Connection': 'close', 'Content-Type': 'multipart/form-data; boundary=a462395550384aa6aee6bbc212c8ea4b', 'Content-Length': '1054368'}

## Reading the returned result

In [9]:
res = decoder.MultipartDecoder.from_response(r)

In [10]:
results = {}
images = {}

for part in res.parts:
    # part is a BodyPart object with b'Content-Type', and b'Content-Disposition', the later includes 'name' and 'filename' info
    headers = {}
    for k, v in part.headers.items():
        headers[k.decode(part.encoding)] = v.decode(part.encoding)

    if headers.get('Content-Type', None) == 'image/jpeg':
        #images[part.headers['filename']] = part.content
        c = headers.get('Content-Disposition')
        image_name = c.split('name="')[1].split('"')[0]  # somehow all the filename and name info is all in one string with no obvious forma
        image = Image.open(io.BytesIO(part.content))
        images[image_name] = image
    elif headers.get('Content-Type', None) == 'application/json':
        content_disposition = headers.get('Content-Disposition', '')
        if 'detection_result' in content_disposition:
            results['detection_result'] = json.loads(part.content.decode())
        elif 'classification_result' in content_disposition:
            results['classification_result'] = json.loads(part.content.decode())

In [11]:
results

{'detection_result': {'S1_D04_R6_PICT0020.JPG': [[0.006578,
    0,
    0.9797779999999999,
    0.9401,
    0.998,
    1]],
  'S1_D04_R6_PICT0021.JPG': [[0.01237, 0, 0.93057, 0.9344, 0.998, 1]]},
 'classification_result': {}}

In [ ]:
for img_name, img in sorted(images.items()):
    print(img_name)
    img
    img.close()  # close after displaying
    print()